# Трассировщик лучей

In [289]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

w = 600
h = 400

# Положение и цвет источника
L = np.array([3., 3., -6.])
color_light = np.ones(3)

# Свойства материалов по умолчанию
ambient = .1
diffuse_c = 1. #k_d

specular_c = 1. #k_s
specular_k = 50 #alpha
refract_c = 1

depth_max = 5 # максимальное количество отражений.
col = np.zeros(3) # текущий цвет.
O = np.array([0., 0.35, -1.]) # Положение камеры.
Q = np.array([0., 0., 0.]) # Направление взгляда камеры.

def normalize(x):
    return x / np.linalg.norm(x)
# луч: из точки O с единичным направляющим вектором D
# плоскость: содержит P и с единичной нормалью N

def intersect_plane(O, D, P, N):
    denom = np.dot(D, N)
    if np.abs(denom) < 1e-6:
        return np.inf
    d = np.dot(P - O, N) / denom
    return np.inf if d < 0 else d

def intersect_sphere(O, D, S, R):
    a = np.dot(D, D)
    OS = O - S
    b = 2 * np.dot(D, OS)
    c = np.dot(OS, OS) - R * R
    disc = b * b - 4 * a * c
    if disc > 0:
        distSqrt = np.sqrt(disc)
        q = (-b - distSqrt) / 2.0 if b < 0 else (-b + distSqrt) / 2.0
        t0 = q / a
        t1 = c / q
        t0, t1 = min(t0, t1), max(t0, t1)
        if t1 >= 0:
            return t1 if t0 < 0 else t0
    return np.inf

def intersect_rect(O, D, P, N):
    denom = np.dot(D, N)
    if np.abs(denom) < 1e-6:
        return np.inf
    d = np.dot(P[0] - O, N) / denom
    if d < 0:
        return np.inf
    M = O + d*D
    'проверка на принадлежность части плоскости - треугольнику'
    A = np.cross(P[0]-M, P[1]-M)
    B = np.cross(P[1]-M, P[2]-M)
    C = np.cross(P[2]-M, P[0]-M)
    if np.dot(A, N) >= 0 and np.dot(B, N) >= 0 and np.dot(C, N) >= 0 or np.dot(A, N) < 0 and np.dot(B, N) < 0 and np.dot(C, N) < 0:
        return d
    return np.inf

def get_normal(obj, M):
    if obj['type'] == 'sphere':
        return normalize(M - obj['position'])
    elif obj['type'] == 'plane':
        return obj['normal']
    elif obj['type'] == 'rect':
        return obj['normal']

def intersect(O, D, obj):
    if obj['type'] == 'plane':
        return intersect_plane(O, D, obj['position'], obj['normal'])
    elif obj['type'] == 'sphere':
        return intersect_sphere(O, D, obj['position'], obj['radius'])
    elif obj['type'] == 'rect':
        return intersect_rect(O, D, obj['position'], obj['normal'])

def get_color(obj, M):
    color = obj['color']
    if not hasattr(color, '__len__'):
        color = color(M)
    return color

def add_sphere(position, radius, color, transparency):
    return dict(type='sphere', position=np.array(position),
        radius=np.array(radius),
        transparent=transparency,
        color=np.array(color), reflection=0.75, refract_c=0.5)

def add_plane(position, normal, transparency):
    return dict(type='plane', position=np.array(position),
        normal=np.array(normal),
        transparent=transparency,
        color=lambda M: (color_plane0
            if (int(M[0] * 2) % 2) == (int(M[2] * 2) % 2)
            else color_plane1),
        diffuse_c=.75, specular_c=.5, reflection=0.5, refract_c=0.5)

def add_rect(position, color, transparency):
    P = np.array(position)
    return dict(type='rect', position=P,
        normal=normalize(np.cross(P[0] - P[1], P[0] - P[2])),
        color=np.array(color),
        transparent=transparency,
        diffuse_c=.75, specular_c=.5, reflection=0.5, refract_c=0.5)

color_plane0 = 1. * np.ones(3)
color_plane1 = 0. * np.ones(3)

#[вправо, вверх, вдаль]
#последний коэффициент - коэффициент преломления(>=1) или 0

scene = [add_sphere([-1.5, 0.5, 1.25], .6, [0., 0., 1.], 0),
        add_sphere([-.5, 0., 2.25], .6, [.1, 1., .1], 1.5),
        add_plane([0., -.75, 0.], [0., 1., 0.], 0),
        add_rect([[1., 0., 1.],[0., 0., 0.],[0., 1., 1.]], [1., 1., 1.], 1.5),
        #add_rect([[1., 0., 1.05],[0., 0., 0.05],[0., 1., 1.05]], [1., .573, .184], 1.5), #призма с первым треугольником
        add_rect([[0., 0., 0.],[-.7, 0., 1.],[0., 1., 1.]], [1., 1., 1.], 1.5),
        add_rect([[0., 0., 0.],[-.7, 0., 1.],[1., 0., 1.]], [1., 1., 1.], 1.5), #грани пирамидки
        add_rect([[-.7, 0., 1.],[0., 1., 1.],[1., 0., 1.]], [1., 1., 1.], 1.5)
        ]

# Картинка
img = np.zeros((h, w, 3))
r = float(w) / h
# Экранные координаты: x0, y0, x1, y1.
S = (-1., -1. / r + .25, 1., 1. / r + .25)

def trace_ray(rayO, rayD, depth=0):
    if depth > depth_max:
        return
    # Первая точка пересечения со сценой.
    t = np.inf
    for i, obj in enumerate(scene):
        t_obj = intersect(rayO, rayD, obj)
        if t_obj < t:
            t, obj_idx = t_obj, i
    if t == np.inf:
        return
    obj = scene[obj_idx]
    # Находим точку пересечения.
    M = rayO + rayD * t
    
    N = get_normal(obj, M)
    color = get_color(obj, M)
    toL = normalize(L - M)
    toO = normalize(O - M)
    
    col_refract = np.zeros(3)
    transp = obj.get('transparent', 0)
    refr = obj.get('refract_c', refract_c)

    if transp != 0:
        if np.dot(rayD, N) >= 0:
            #закон Снеллиуса - выходящий луч
            sin_rayD = np.sqrt(1 - np.dot(rayD, N)**2)
            sin_res = sin_rayD * transp
            ang_rayD = np.arcsin(sin_rayD)
            ang_res = np.arcsin(sin_res)
            sin_dif = np.sin(ang_rayD - ang_res)
            
            rayO = M + N * .0001
            rayD = rayD * sin_res/sin_rayD + N*sin_dif/sin_rayD
        else:
            #закон Снеллиуса - входящий луч
            sin_rayD = np.sqrt(1 - np.dot(rayD, -N)**2)
            sin_res = sin_rayD / transp
            ang_rayD = np.arcsin(sin_rayD)
            ang_res = np.arcsin(sin_res)
            sin_dif = np.sin(ang_rayD - ang_res)
            
            rayO = M - N * .0001
            rayD = rayD * sin_res/sin_rayD - N*sin_dif/sin_rayD
        traced = trace_ray(rayO, rayD, depth)
        if traced:
            obj, M, N, col_ray = traced
            depth += 1
            col_refract = col_ray    
    
    # Цвет = ambient + diffuse(Lambert) + specular(Blinn-Phong).
    col_ray = ambient
    col_ray += (obj.get('diffuse_c', diffuse_c)
        * max(np.dot(N, toL), 0) * color)
    col_ray += (obj.get('specular_c', specular_c)
        * max(np.dot(N, normalize(toL + toO)), 0) ** specular_k
        * color_light)
    col_ray += obj.get('refract_c', refract_c) * col_refract
    
    # В тени или нет? Тень прозрачного объекта серая, а непрозрачного чёрная
    l = [intersect(M + N * .0001, toL, obj_sh)
        for k, obj_sh in enumerate(scene) if k != obj_idx]
    if l and min(l) < np.inf:
        if l.index(min(l)) >= obj_idx:
            transp = scene[l.index(min(l))-1].get('transparent', 0.)
        else:
            transp = scene[l.index(min(l))].get('transparent', 0.)
        if transp != 0:
            col_ray *= 0.75
        else:
            col_ray *= 0.
    return obj, M, N, col_ray

for i, x in enumerate(np.linspace(S[0], S[2], w)):
    if i % 50 == 0:
        print (round(i / float(w) * 100,2), '%')
    for j, y in enumerate(np.linspace(S[1], S[3], h)):
        col[:] = 0
        Q[:2] = (x, y)
        D = normalize(Q - O)
        depth = 0
        rayO, rayD = O, D
        reflection = 1.
        # Цикл по лучам.
        while depth < depth_max:
                traced = trace_ray(rayO, rayD, depth)
                if not traced:
                    break
                obj, M, N, col_ray = traced
                # Создание луча в случае отражения.
                if np.dot(rayD, N) >= 0:
                    rayO = M - N * .0001
                else:
                    rayO = M + N * .0001
                rayD = normalize(rayD - 2 * np.dot(rayD, N) * N)
                depth += 1
                col += reflection * col_ray
                reflection *= obj.get('reflection', 1.)
        img[h - j - 1, i, :] = np.clip(col, 0, 1)

plt.imsave('fig.png', img)

0.0 %
8.33 %
16.67 %


/home/liza/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:170: RuntimeWarning: invalid value encountered in arcsin


25.0 %
33.33 %
41.67 %
50.0 %
58.33 %
66.67 %
75.0 %
83.33 %
91.67 %
